# 5.2. Pandas: Data Wrangling

Module M-227-04: Programming for Data Analytics

Instructor: prof. Dmitry Pavlyuk

## Data manipulation

* Hierarchical indexing (multi-index)
* Data reshaping
* Joining data frames
* Data aggregation

## Example data set: Students

4 dataframes:
* __courses__ stores information about available courses
* __groups__ stores information about student groups
* __students__ stores information individual students
* __results__ stores information about course attendance by students and grades

## Example data set

* Loading __courses__

In [125]:
import pandas as pd
import numpy as np
courses_df = pd.read_csv('data/st_courses.csv') 
courses_df.head()

,course_id,course_name
0,1,Information Systems and Technologies
1,2,Mathematics for data analytics
2,3,Modern Database Technologies
3,4,Programming for Data Analytics
4,5,Advanced Artificial Intelligence


* Loading __groups__

In [33]:
groups_df = pd.read_csv('data/st_groups.csv') 
groups_df.head()

,group_id,group_name,group_year_started
0,1,4101MDA,2021
1,2,4201MDA,2022
2,3,4203MDA,2022


## Example data set

* Loading __students__

In [34]:
students_df = pd.read_csv('data/st_students.csv') 
students_df.head()

,student_id,lastname,firstname,group_id
0,101,M,Jurijs,2
1,102,J,Kaspars,1
2,103,Z,Jānis,3
3,104,P,Iļja,2
4,105,Z,Andris,2


* Loading __results__

In [35]:
results_df = pd.read_csv('data/st_results.csv') 
results_df.head()

,course_id,student_id,attendance,grade
0,1,103,55,NaN
1,1,106,60,NaN
2,1,109,40,NaN
3,1,110,80,NaN
4,1,111,60,NaN


## Hierarchical indexing

### Recall indexes

We already discussed one-sequence indexes in the previous presentation:

In [36]:
students_df.set_index("student_id", inplace=True)
students_df.head()

,lastname,firstname,group_id
student_id,,,
101,M,Jurijs,2
102,J,Kaspars,1
103,Z,Jānis,3
104,P,Iļja,2
105,Z,Andris,2


In [37]:
groups_df.set_index("group_id", inplace=True)
courses_df.set_index("course_id", inplace=True)

### Hierarchical indexing

Indexes in pandas data frames are not limited by one sequence:

In [38]:
results_df = results_df.set_index(["course_id","student_id"]).sort_index()
results_df.head(18)

attendance  grade
course_id student_id                   
1         103                 55    NaN
          106                 60    NaN
          109                 40    NaN
          110                 80    NaN
          111                 60    NaN
          112                 80    NaN
2         103                 35    NaN
          106                 40    NaN
          109                 40    NaN
          110                 70    NaN
          111                 40    NaN
          112                 40    NaN
3         103                 51    NaN
          106                 61    NaN
          109                 71    NaN
          110                 51    NaN
          111                 41    NaN
          112                 31    NaN

### Multi-index structure

Indexes in pandas data frames are not limited by one sequence:

In [39]:
results_df.index.nlevels

2

In [40]:
results_df.index

MultiIndex([(1, 103),
            (1, 106),
            (1, 109),
            (1, 110),
            (1, 111),
            (1, 112),
            (2, 103),
            (2, 106),
            (2, 109),
            (2, 110),
            (2, 111),
            (2, 112),
            (3, 103),
            (3, 106),
            (3, 109),
            (3, 110),
            (3, 111),
            (3, 112),
            (5, 101),
            (5, 102),
            (5, 104),
            (5, 105),
            (5, 107),
            (5, 108),
            (5, 113),
            (5, 114)],
           names=['course_id', 'student_id'])

### Hierarchical indexing

Two-level index for __students__:

In [41]:
students_df = students_df.reset_index().set_index(["group_id","student_id"]).sort_index()
students_df

lastname   firstname
group_id student_id                     
1        102               J     Kaspars
         107               J       Ahmed
         113               I      Simeon
         114               K      Oskars
2        101               M      Jurijs
         104               P        Iļja
         105               Z      Andris
         108               K    Tamanjit
3        103               Z       Jānis
         106               Z      Ņikita
         109               K      Alexey
         110               M  Vjačeslavs
         111               B   Jevgenijs
         112               T       Ērika

### Hierarchical indexing: selectors

In [42]:
results_df.loc[1] # Selecting by the first-level, course

,attendance,grade
student_id,,
103,55,NaN
106,60,NaN
109,40,NaN
110,80,NaN
111,60,NaN
112,80,NaN


In [43]:
results_df.loc[:,103,:] # Selecting by the second-level, student

,attendance,grade
course_id,,
1,55,NaN
2,35,NaN
3,51,NaN


## Reshaping data

### Rearrange levels

* Change order of levels (__swap__)

In [44]:
results_df.swaplevel("student_id", "course_id").sort_index().head(10)

attendance  grade
student_id course_id                   
101        5                  60    NaN
102        5                  95    7.0
103        1                  55    NaN
           2                  35    NaN
           3                  51    NaN
104        5                  70    NaN
105        5                  50    NaN
106        1                  60    NaN
           2                  40    NaN
           3                  61    NaN

### Index to column

* Convert an index level to a column (__reset__)

In [45]:
results_df.reset_index("student_id").head()

,student_id,attendance,grade
course_id,,,
1,103,55,NaN
1,106,60,NaN
1,109,40,NaN
1,110,80,NaN
1,111,60,NaN


### Row index to column index

* Transfer a level from the row index to the column index (__unstack__)

In [46]:
results_df.unstack()

attendance                                                        \
student_id        101   102   103   104   105   106   107   108   109   110   
course_id                                                                     
1                 NaN   NaN  55.0   NaN   NaN  60.0   NaN   NaN  40.0  80.0   
2                 NaN   NaN  35.0   NaN   NaN  40.0   NaN   NaN  40.0  70.0   
3                 NaN   NaN  51.0   NaN   NaN  61.0   NaN   NaN  71.0  51.0   
5                60.0  95.0   NaN  70.0  50.0   NaN  65.0  70.0   NaN   NaN   

            ... grade                                        
student_id  ...   105 106 107 108 109 110 111 112  113  114  
course_id   ...                                              
1           ...   NaN NaN NaN NaN NaN NaN NaN NaN  NaN  NaN  
2           ...   NaN NaN NaN NaN NaN NaN NaN NaN  NaN  NaN  
3           ...   NaN NaN NaN NaN NaN NaN NaN NaN  NaN  NaN  
5           ...   NaN NaN NaN NaN NaN NaN NaN NaN  8.0  9.0  

[4 rows x 28 columns]

In [47]:
df1 = results_df.unstack(level=0)
df1.head()

attendance                   grade             
course_id           1     2     3     5     1   2   3    5
student_id                                                
101               NaN   NaN   NaN  60.0   NaN NaN NaN  NaN
102               NaN   NaN   NaN  95.0   NaN NaN NaN  7.0
103              55.0  35.0  51.0   NaN   NaN NaN NaN  NaN
104               NaN   NaN   NaN  70.0   NaN NaN NaN  NaN
105               NaN   NaN   NaN  50.0   NaN NaN NaN  NaN

### Column index to row index

* Transfer a level back from the column index to the row index (__stack__)

In [48]:
df1.stack().head()

attendance  grade
student_id course_id                   
101        5                60.0    NaN
102        5                95.0    7.0
103        1                55.0    NaN
           2                35.0    NaN
           3                51.0    NaN

### Long data

Potentially, we can move all column index levels into rows, so every row will contain only a single observation:

In [49]:
results_df.columns.name="indicator"
results_df.stack()

course_id  student_id  indicator 
1          103         attendance    55.0
           106         attendance    60.0
           109         attendance    40.0
           110         attendance    80.0
           111         attendance    60.0
           112         attendance    80.0
2          103         attendance    35.0
           106         attendance    40.0
           109         attendance    40.0
           110         attendance    70.0
           111         attendance    40.0
           112         attendance    40.0
3          103         attendance    51.0
           106         attendance    61.0
           109         attendance    71.0
           110         attendance    51.0
           111         attendance    41.0
           112         attendance    31.0
5          101         attendance    60.0
           102         attendance    95.0
                       grade          7.0
           104         attendance    70.0
           105         attendance    50.0


### Long data - melt

Similarly, we can do the same operation (but for columns, not indexes) using __melt__

In [50]:
results_long = results_df.reset_index().melt(id_vars=["course_id", "student_id"]).dropna()
results_long.head()

,course_id,student_id,indicator,value
0,1,103,attendance,55.0
1,1,106,attendance,60.0
2,1,109,attendance,40.0
3,1,110,attendance,80.0
4,1,111,attendance,60.0


### Long data - reasons

* Efficient storage of sparse data (many NaN values)
* Adding new indicator (value in a column) is possible without changing the table structure
* Iterating over one dimension can be more efficient (thus, many third-party functions require long data)

### Wide data - pivot

__pivot__ operation is an inverse to __melt__ (like __stack/unstack__, but for columns)

In [51]:
results_long.pivot(index=["course_id","student_id"], columns=["indicator"],values="value").head(18)

indicator             attendance  grade
course_id student_id                   
1         103               55.0    NaN
          106               60.0    NaN
          109               40.0    NaN
          110               80.0    NaN
          111               60.0    NaN
          112               80.0    NaN
2         103               35.0    NaN
          106               40.0    NaN
          109               40.0    NaN
          110               70.0    NaN
          111               40.0    NaN
          112               40.0    NaN
3         103               51.0    NaN
          106               61.0    NaN
          109               71.0    NaN
          110               51.0    NaN
          111               41.0    NaN
          112               31.0    NaN

## Joining data frames

### Concatenate

Concatenating data frames along with an axis and performing optional set logic (union or intersection) of the indexes on the other axes.

In [52]:
groups_df

,group_name,group_year_started
group_id,,
1,4101MDA,2021
2,4201MDA,2022
3,4203MDA,2022


In [53]:
groups2_df=pd.DataFrame({'group_id':[4],'group_name':['4202MDA'], 'group_year_started':[2022], 'programme':['MIS']})
groups2_df.set_index('group_id', inplace=True)
groups2_df

,group_name,group_year_started,programme
group_id,,,
4,4202MDA,2022,MIS


### Concatenate

* Concatenating rows and creating __union__ of columns (with NaN if necessary)

In [54]:
pd.concat([groups_df,groups2_df])

,group_name,group_year_started,programme
group_id,,,
1,4101MDA,2021,NaN
2,4201MDA,2022,NaN
3,4203MDA,2022,NaN
4,4202MDA,2022,MIS


* Concatenating rows and creating __intersection__ of columns

In [55]:
pd.concat([groups_df,groups2_df], join="inner")

,group_name,group_year_started
group_id,,
1,4101MDA,2021
2,4201MDA,2022
3,4203MDA,2022
4,4202MDA,2022


### Concatenate

* Concatenating rows and introducing indexes to distinguish data frames

In [56]:
pd.concat([groups_df,groups2_df], join="inner", keys=['g1','g2'])

group_name  group_year_started
   group_id                               
g1 1           4101MDA                2021
   2           4201MDA                2022
   3           4203MDA                2022
g2 4           4202MDA                2022

### Concatenate

* Concatenating columns and creating __union__ of rows

In [57]:
pd.concat([groups_df,groups2_df], axis=1)

,group_name,group_year_started,group_name,group_year_started,programme
group_id,,,,,
1,4101MDA,2021.0,NaN,NaN,NaN
2,4201MDA,2022.0,NaN,NaN,NaN
3,4203MDA,2022.0,NaN,NaN,NaN
4,NaN,NaN,4202MDA,2022.0,MIS


In [58]:
groups_df=pd.concat([groups_df,groups2_df], join="inner").reset_index()

### Merge - database-style joins

* INNER JOIN
* LEFT (OUTER) JOIN
* RIGHT (OUTER) JOIN
* CROSS (FULL) JOIN

<img style="float:left" src="https://www.w3schools.com/sql/img_innerjoin.gif"/>
<img style="float:left;padding-left:20px;" src="https://www.w3schools.com/sql/img_leftjoin.gif"/>
<img style="float:left;padding-left:20px;" src="https://www.w3schools.com/sql/img_rightjoin.gif"/>
<img style="float:left;padding-left:20px;" src="https://www.w3schools.com/sql/img_fulljoin.gif"/>

### Inner join

In [59]:
groups_df.set_index("group_id", inplace=True)
results_df.reset_index(inplace=True)
pd.merge(students_df, groups_df, left_on="group_id", right_index=True)

lastname   firstname group_name  group_year_started
group_id student_id                                                    
1        102               J     Kaspars    4101MDA                2021
         107               J       Ahmed    4101MDA                2021
         113               I      Simeon    4101MDA                2021
         114               K      Oskars    4101MDA                2021
2        101               M      Jurijs    4201MDA                2022
         104               P        Iļja    4201MDA                2022
         105               Z      Andris    4201MDA                2022
         108               K    Tamanjit    4201MDA                2022
3        103               Z       Jānis    4203MDA                2022
         106               Z      Ņikita    4203MDA                2022
         109               K      Alexey    4203MDA                2022
         110               M  Vjačeslavs    4203MDA                2022
         111               B   Jevgenijs    4203MDA                2022
         112               T       Ērika    4203MDA                2022

### Right outer join

In [63]:
pd.merge(students_df, groups_df, left_on="group_id", right_index=True, how="right").tail()

,group_id,lastname,firstname,group_name,group_year_started
"(3, 109)",3,K,Alexey,4203MDA,2022
"(3, 110)",3,M,Vjačeslavs,4203MDA,2022
"(3, 111)",3,B,Jevgenijs,4203MDA,2022
"(3, 112)",3,T,Ērika,4203MDA,2022
NaN,4,NaN,NaN,4202MDA,2022


### Left outer join

In [62]:
pd.merge(students_df, results_df, left_on="student_id", right_on="student_id", how="left").head()

,student_id,lastname,firstname,course_id,attendance,grade
0,102,J,Kaspars,5,95,7.0
1,107,J,Ahmed,5,65,NaN
2,113,I,Simeon,5,90,8.0
3,114,K,Oskars,5,90,9.0
4,101,M,Jurijs,5,60,NaN


## Data aggregation

### Data aggregation

Aggregation refers to the process by which entities are combined to form a single meaningful entity.
Aggregation in Pandas and databases are fairly the same - refer __GROUP BY__ of SQL.

In [95]:
results_df.columns.name=None
results_df.reset_index(inplace=True)
results_df.set_index(["course_id","student_id"], inplace=True)
results_df.head(18)

attendance  grade
course_id student_id                   
1         103                 55    NaN
          106                 60    NaN
          109                 40    NaN
          110                 80    NaN
          111                 60    NaN
          112                 80    NaN
2         103                 35    NaN
          106                 40    NaN
          109                 40    NaN
          110                 70    NaN
          111                 40    NaN
          112                 40    NaN
3         103                 51    NaN
          106                 61    NaN
          109                 71    NaN
          110                 51    NaN
          111                 41    NaN
          112                 31    NaN

### Groupby

__.groupby__ creates a special object for easy access to groups and calculating within-statistics

In [96]:
results_df.groupby("course_id")

In [145]:
list(results_df.groupby("course_id"))[0]

(1,
                       attendance  grade
 course_id student_id                   
 1         103                 55    NaN
           106                 60    NaN
           109                 40    NaN
           110                 80    NaN
           111                 60    NaN
           112                 80    NaN)

### Aggregation

* Statistical - max(), min(), mean(), sum(), std(), count()
* Element selection - first(), last(), head(), tail()
* Custom function  - agg()

### Statistical

In [98]:
results_df.groupby("course_id").mean()

,attendance,grade
course_id,,
1,62.500000,NaN
2,44.166667,NaN
3,51.000000,NaN
5,73.750000,8.0


In [99]:
results_df.groupby("course_id").count()

,attendance,grade
course_id,,
1,6,0
2,6,0
3,6,0
5,8,3


### Element selection

In [101]:
results_df.sort_values("attendance").groupby("course_id").last()

,attendance,grade
course_id,,
1,80,NaN
2,70,NaN
3,71,NaN
5,95,7.0


In [102]:
results_df.sort_values("attendance").groupby("course_id").first()

,attendance,grade
course_id,,
1,40,NaN
2,35,NaN
3,31,NaN
5,50,8.0


### Custom function

In [104]:
results_df.groupby("course_id").agg(lambda x: max(x)- min(x))

,attendance,grade
course_id,,
1,40,NaN
2,35,NaN
3,40,NaN
5,45,NaN


In [121]:
def myrange(x):
    return max(x)- min(x)
results_df.groupby("course_id").agg([min, max,myrange])

attendance             grade             
                 min max myrange   min  max myrange
course_id                                          
1                 40  80      40   NaN  NaN     NaN
2                 35  70      35   NaN  NaN     NaN
3                 31  71      40   NaN  NaN     NaN
5                 50  95      45   7.0  9.0     NaN

### Custom function

Flexible set of functions

In [132]:
results_df.groupby("course_id").agg({"attendance":[min, np.mean], "grade":["count"]})

attendance            grade
                 min       mean count
course_id                            
1                 40  62.500000     0
2                 35  44.166667     0
3                 31  51.000000     0
5                 50  73.750000     3

Note how functions can be passed - __min__, __np_mean__, __"count"__

# Thank you